# GCP Demos

### Capstone Project GCP Location: https://console.cloud.google.com/home/dashboard?project=project2-2019-259821

### Geo info: http://donnees.ville.montreal.qc.ca/dataset/geobase
https://robertorocha.info/how-i-made-the-montreal-street-history-map/

In [1]:
import networkx as nx
import pandas as pd
import requests
import json
from datetime import datetime
import time
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open(r"./geojson_dump.json", 'r') as f:    
    data = f.read()

In [3]:
data_json = json.loads(data)

In [4]:
def distance_in_km(e1, e2):

    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(e1[1])
    lon1 = radians(e1[0])
    lat2 = radians(e2[1])
    lon2 = radians(e2[0])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [66]:
def get_flow_rate(lat, long):
    """
    Returns free flow traffic speed at a given coordinate in KMPH
    """
    CLIENT_KEY = ''
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key={}&point={},{}&unit=KMPH&jsonp=jsonp".format(CLIENT_KEY,
                                                                                                               lat, long)
    r = requests.get(url)
    if r.status_code != 200:
        err = json.loads(r.content)
        if 'error' in err.keys():
            if err['error'] == "Point too far from nearest existing segment.":
                return 1
        raise ValueError("Invalid URL Response for point {},{}".format(lat, long))
    return json.loads(r.content)['flowSegmentData']['freeFlowSpeed']

In [6]:
d = []
for features in data_json['features']:
    d.append(features['properties']['ARR_DRT'])
print(set(d))

{'Le Plateau-Mont-Royal', 'Le Sud-Ouest', 'Rosemont-La Petite-Patrie', 'Ville-Marie', 'Outremont', 'N/A', 'Ahuntsic-Cartierville', 'Anjou', 'LaSalle', 'Montréal-Nord', 'Saint-Laurent', "L'Île-Bizard-Sainte-Geneviève", 'Saint-Léonard', 'Lachine', 'Côte-des-Neiges-Notre-Dame-de-Grâce', 'Mercier-Hochelaga-Maisonneuve', 'Rivière-des-Prairies-Pointe-aux-Trembles', 'Verdun', 'Villeray-Saint-Michel-Parc-Extension', 'Pierrefonds-Roxboro'}


In [7]:
def middle_point(l):
    return l[int((len(l)-1)/2)][1], l[int((len(l)-1)/2)][0]

In [13]:
def diedge_exists(a, b):
    # direction matters!
    try:
        return b in G.neighbors(a)
    except nx.NetworkXError:
        return False

In [62]:
existing_nodes = [x[0] for x in list(G.nodes(data=True))]
existing_set = set(existing_nodes)

In [ ]:
G = nx.DiGraph()

In [63]:
for j, feature in enumerate(data_json['features']):
    
    feature_set = set([tuple(x) for x in feature['geometry']['coordinates']])
    if len(feature_set - existing_set) == 0:
        continue
        
    sens_cir = feature['properties']['SENS_CIR']
    middle = middle_point(feature['geometry']['coordinates'])
    
    print(j, '/', len(data_json['features']))
    rate = max(1, get_flow_rate(*middle))
    
    for idx, coord in enumerate(feature['geometry']['coordinates']):
        coord = tuple(coord)
        if coord not in G:
            G.add_node(coord, **feature['properties'])
        if idx != 0:
            dist = distance_in_km(coord, last_node)
            tm = dist / rate
            if sens_cir == 0:
                G.add_edge(last_node, coord, length=dist, time=tm)
                G.add_edge(coord, last_node, length=dist, time=tm)
            elif sens_cir == 1:
                G.add_edge(last_node, coord, length=dist, time=tm)
            elif sens_cir == -1:
                G.add_edge(coord, last_node, length=dist, time=tm)
        last_node = coord

17841 / 47391
17842 / 47391
17843 / 47391
17844 / 47391
17845 / 47391
17846 / 47391
17847 / 47391
17848 / 47391
17849 / 47391
17850 / 47391
17851 / 47391
17852 / 47391
17853 / 47391
17854 / 47391
17855 / 47391
17856 / 47391
17857 / 47391
17858 / 47391
17859 / 47391
17860 / 47391
17861 / 47391
17862 / 47391
17863 / 47391
17864 / 47391
17865 / 47391
17866 / 47391
17867 / 47391
17868 / 47391
17869 / 47391
17870 / 47391
17871 / 47391
17872 / 47391
17873 / 47391
17874 / 47391
17875 / 47391
17876 / 47391
17877 / 47391
17878 / 47391
17879 / 47391
17880 / 47391
17881 / 47391
17882 / 47391
17883 / 47391
17884 / 47391
17885 / 47391
17886 / 47391
17887 / 47391
17888 / 47391
17889 / 47391
17890 / 47391
17891 / 47391
17892 / 47391
17893 / 47391
17894 / 47391
17895 / 47391
17896 / 47391
17897 / 47391
17898 / 47391
17899 / 47391
17900 / 47391
17901 / 47391
17902 / 47391
17903 / 47391
17904 / 47391
17905 / 47391
17906 / 47391
17907 / 47391
17908 / 47391
17909 / 47391
17910 / 47391
17911 / 47391
17912 

18471 / 47391
18472 / 47391
18473 / 47391
18474 / 47391
18475 / 47391
18476 / 47391
18477 / 47391
18478 / 47391
18479 / 47391
18480 / 47391
18481 / 47391
18482 / 47391
18483 / 47391
18484 / 47391
18485 / 47391
18486 / 47391
18487 / 47391
18488 / 47391
18489 / 47391
18490 / 47391
18491 / 47391
18492 / 47391
18493 / 47391
18494 / 47391
18495 / 47391
18496 / 47391
18497 / 47391
18498 / 47391
18499 / 47391
18500 / 47391
18501 / 47391
18502 / 47391
18503 / 47391
18504 / 47391
18505 / 47391
18506 / 47391
18507 / 47391
18508 / 47391
18509 / 47391
18510 / 47391
18511 / 47391
18512 / 47391
18513 / 47391
18514 / 47391
18515 / 47391
18516 / 47391
18517 / 47391
18518 / 47391
18519 / 47391
18520 / 47391
18521 / 47391
18522 / 47391
18523 / 47391
18524 / 47391
18525 / 47391
18526 / 47391
18527 / 47391
18528 / 47391
18529 / 47391
18530 / 47391
18531 / 47391
18532 / 47391
18533 / 47391
18534 / 47391
18535 / 47391
18536 / 47391
18537 / 47391
18538 / 47391
18539 / 47391
18540 / 47391
18541 / 47391
18542 

19057 / 47391
19058 / 47391
19059 / 47391
19060 / 47391
19061 / 47391
19062 / 47391
19063 / 47391
19064 / 47391
19065 / 47391
19066 / 47391
19067 / 47391
19068 / 47391
19069 / 47391
19070 / 47391
19071 / 47391
19072 / 47391
19073 / 47391
19074 / 47391
19075 / 47391
19076 / 47391
19077 / 47391
19078 / 47391
19079 / 47391
19080 / 47391
19081 / 47391
19082 / 47391
19083 / 47391
19084 / 47391
19085 / 47391
19086 / 47391
19087 / 47391
19088 / 47391
19089 / 47391
19090 / 47391
19091 / 47391
19092 / 47391
19093 / 47391
19094 / 47391
19095 / 47391
19096 / 47391
19097 / 47391
19098 / 47391
19099 / 47391
19100 / 47391
19101 / 47391
19102 / 47391
19103 / 47391
19104 / 47391
19105 / 47391
19106 / 47391
19107 / 47391
19108 / 47391
19109 / 47391
19110 / 47391
19111 / 47391
19112 / 47391
19113 / 47391
19114 / 47391
19115 / 47391
19116 / 47391
19117 / 47391
19118 / 47391
19119 / 47391
19120 / 47391
19121 / 47391
19122 / 47391
19123 / 47391
19124 / 47391
19125 / 47391
19126 / 47391
19127 / 47391
19128 

19643 / 47391
19644 / 47391
19645 / 47391
19646 / 47391
19647 / 47391
19648 / 47391
19649 / 47391
19650 / 47391
19651 / 47391
19652 / 47391
19653 / 47391
19654 / 47391
19655 / 47391
19656 / 47391
19657 / 47391
19658 / 47391
19659 / 47391
19660 / 47391
19661 / 47391
19662 / 47391
19663 / 47391
19664 / 47391
19665 / 47391
19666 / 47391
19667 / 47391
19668 / 47391
19669 / 47391
19670 / 47391
19671 / 47391
19672 / 47391
19673 / 47391
19674 / 47391
19675 / 47391
19676 / 47391
19677 / 47391
19678 / 47391
19679 / 47391
19680 / 47391
19681 / 47391
19682 / 47391
19683 / 47391
19684 / 47391
19685 / 47391
19686 / 47391
19687 / 47391
19688 / 47391
19689 / 47391
19690 / 47391
19691 / 47391
19692 / 47391
19693 / 47391
19694 / 47391
19695 / 47391
19696 / 47391
19697 / 47391
19698 / 47391
19699 / 47391
19700 / 47391
19701 / 47391
19702 / 47391
19703 / 47391
19704 / 47391
19705 / 47391
19706 / 47391
19707 / 47391
19708 / 47391
19709 / 47391
19710 / 47391
19711 / 47391
19712 / 47391
19713 / 47391
19714 

20236 / 47391
20237 / 47391
20238 / 47391
20239 / 47391
20240 / 47391
20241 / 47391
20242 / 47391
20243 / 47391
20244 / 47391
20245 / 47391
20246 / 47391
20247 / 47391
20248 / 47391
20249 / 47391
20250 / 47391
20251 / 47391
20252 / 47391
20253 / 47391
20254 / 47391
20255 / 47391
20256 / 47391
20257 / 47391
20258 / 47391
20259 / 47391
20260 / 47391
20261 / 47391
20262 / 47391
20263 / 47391
20264 / 47391
20265 / 47391
20266 / 47391
20267 / 47391
20268 / 47391
20269 / 47391
20270 / 47391
20271 / 47391
20272 / 47391
20273 / 47391
20274 / 47391
20275 / 47391
20276 / 47391
20277 / 47391
20278 / 47391
20279 / 47391
20280 / 47391
20281 / 47391
20282 / 47391
20283 / 47391
20284 / 47391
20285 / 47391
20286 / 47391
20287 / 47391
20288 / 47391
20289 / 47391
20290 / 47391
20291 / 47391
20292 / 47391
20293 / 47391
20294 / 47391
20295 / 47391
20296 / 47391
20297 / 47391
20298 / 47391
20299 / 47391
20300 / 47391
20301 / 47391
20302 / 47391
20303 / 47391
20304 / 47391
20305 / 47391
20306 / 47391
20307 

...

44879 / 47391
44880 / 47391
44881 / 47391
44882 / 47391
44883 / 47391
44884 / 47391
44885 / 47391
44886 / 47391
44887 / 47391
44888 / 47391
44889 / 47391
44890 / 47391
44891 / 47391
44892 / 47391
44893 / 47391
44894 / 47391
44895 / 47391
44896 / 47391
44897 / 47391
44898 / 47391
44899 / 47391
44900 / 47391
44901 / 47391
44902 / 47391
44903 / 47391
44904 / 47391
44905 / 47391
44906 / 47391
44907 / 47391
44908 / 47391
44909 / 47391
44910 / 47391
44911 / 47391
44912 / 47391
44913 / 47391
44914 / 47391
44915 / 47391
44916 / 47391
44917 / 47391
44918 / 47391
44919 / 47391
44920 / 47391
44921 / 47391
44922 / 47391
44923 / 47391
44924 / 47391
44925 / 47391
44926 / 47391
44927 / 47391
44928 / 47391
44929 / 47391
44930 / 47391
44931 / 47391
44932 / 47391
44933 / 47391
44934 / 47391
44935 / 47391
44936 / 47391
44937 / 47391
44938 / 47391
44939 / 47391
44940 / 47391
44941 / 47391
44942 / 47391
44943 / 47391
44944 / 47391
44945 / 47391
44946 / 47391
44947 / 47391
44948 / 47391
44949 / 47391
44950 

45466 / 47391
45467 / 47391
45468 / 47391
45469 / 47391
45470 / 47391
45471 / 47391
45472 / 47391
45473 / 47391
45474 / 47391
45475 / 47391
45476 / 47391
45477 / 47391
45478 / 47391
45479 / 47391
45480 / 47391
45481 / 47391
45482 / 47391
45483 / 47391
45484 / 47391
45485 / 47391
45486 / 47391
45487 / 47391
45488 / 47391
45489 / 47391
45490 / 47391
45491 / 47391
45492 / 47391
45493 / 47391
45494 / 47391
45495 / 47391
45496 / 47391
45497 / 47391
45498 / 47391
45499 / 47391
45500 / 47391
45501 / 47391
45502 / 47391
45503 / 47391
45504 / 47391
45505 / 47391
45506 / 47391
45507 / 47391
45508 / 47391
45509 / 47391
45510 / 47391
45511 / 47391
45512 / 47391
45513 / 47391
45514 / 47391
45515 / 47391
45516 / 47391
45517 / 47391
45518 / 47391
45519 / 47391
45520 / 47391
45521 / 47391
45522 / 47391
45523 / 47391
45524 / 47391
45525 / 47391
45526 / 47391
45527 / 47391
45528 / 47391
45529 / 47391
45530 / 47391
45531 / 47391
45532 / 47391
45533 / 47391
45534 / 47391
45535 / 47391
45536 / 47391
45537 

46052 / 47391
46053 / 47391
46054 / 47391
46055 / 47391
46056 / 47391
46057 / 47391
46058 / 47391
46059 / 47391
46060 / 47391
46061 / 47391
46062 / 47391
46063 / 47391
46064 / 47391
46065 / 47391
46066 / 47391
46067 / 47391
46068 / 47391
46069 / 47391
46070 / 47391
46071 / 47391
46072 / 47391
46073 / 47391
46074 / 47391
46075 / 47391
46076 / 47391
46077 / 47391
46078 / 47391
46079 / 47391
46080 / 47391
46081 / 47391
46082 / 47391
46083 / 47391
46084 / 47391
46085 / 47391
46086 / 47391
46087 / 47391
46088 / 47391
46089 / 47391
46090 / 47391
46091 / 47391
46092 / 47391
46093 / 47391
46094 / 47391
46095 / 47391
46096 / 47391
46097 / 47391
46098 / 47391
46099 / 47391
46100 / 47391
46101 / 47391
46102 / 47391
46103 / 47391
46104 / 47391
46105 / 47391
46106 / 47391
46107 / 47391
46108 / 47391
46109 / 47391
46110 / 47391
46111 / 47391
46112 / 47391
46113 / 47391
46114 / 47391
46115 / 47391
46116 / 47391
46117 / 47391
46118 / 47391
46119 / 47391
46120 / 47391
46121 / 47391
46122 / 47391
46123 

46638 / 47391
46639 / 47391
46640 / 47391
46641 / 47391
46642 / 47391
46643 / 47391
46644 / 47391
46645 / 47391
46646 / 47391
46647 / 47391
46648 / 47391
46649 / 47391
46650 / 47391
46651 / 47391
46652 / 47391
46653 / 47391
46654 / 47391
46655 / 47391
46656 / 47391
46657 / 47391
46658 / 47391
46659 / 47391
46660 / 47391
46661 / 47391
46662 / 47391
46663 / 47391
46664 / 47391
46665 / 47391
46666 / 47391
46667 / 47391
46668 / 47391
46669 / 47391
46670 / 47391
46671 / 47391
46672 / 47391
46673 / 47391
46674 / 47391
46675 / 47391
46676 / 47391
46677 / 47391
46678 / 47391
46679 / 47391
46680 / 47391
46681 / 47391
46682 / 47391
46683 / 47391
46684 / 47391
46685 / 47391
46686 / 47391
46687 / 47391
46688 / 47391
46689 / 47391
46690 / 47391
46691 / 47391
46692 / 47391
46693 / 47391
46694 / 47391
46695 / 47391
46696 / 47391
46697 / 47391
46698 / 47391
46699 / 47391
46700 / 47391
46701 / 47391
46702 / 47391
46703 / 47391
46704 / 47391
46705 / 47391
46706 / 47391
46707 / 47391
46708 / 47391
46709 

47226 / 47391
47227 / 47391
47228 / 47391
47229 / 47391
47230 / 47391
47231 / 47391
47232 / 47391
47233 / 47391
47234 / 47391
47235 / 47391
47236 / 47391
47237 / 47391
47238 / 47391
47239 / 47391
47241 / 47391
47242 / 47391
47243 / 47391
47244 / 47391
47245 / 47391
47246 / 47391
47247 / 47391
47248 / 47391
47249 / 47391
47250 / 47391
47251 / 47391
47252 / 47391
47253 / 47391
47254 / 47391
47255 / 47391
47256 / 47391
47257 / 47391
47258 / 47391
47260 / 47391
47261 / 47391
47262 / 47391
47263 / 47391
47264 / 47391
47265 / 47391
47266 / 47391
47267 / 47391
47268 / 47391
47269 / 47391
47270 / 47391
47271 / 47391
47272 / 47391
47273 / 47391
47274 / 47391
47275 / 47391
47276 / 47391
47277 / 47391
47278 / 47391
47279 / 47391
47280 / 47391
47281 / 47391
47282 / 47391
47283 / 47391
47284 / 47391
47285 / 47391
47286 / 47391
47287 / 47391
47288 / 47391
47289 / 47391
47292 / 47391
47293 / 47391
47294 / 47391
47295 / 47391
47296 / 47391
47297 / 47391
47298 / 47391
47299 / 47391
47300 / 47391
47301 

In [65]:
nx.write_gpickle(G,"full_graph.gpickle")

In [67]:
def longest_node_distance():
    max_dist = 0
    for idx, start in enumerate(list(G.nodes(data=True))):
        for end in list(G.nodes(data=True)):
            if start != end:
                try:
                    dist = nx.shortest_path_length(G,source=start[0], target=end[0], weight='length')
                except nx.NetworkXNoPath:
                    continue
                if dist > max_dist:
                    max_dist = dist
                    start_node = start
                    end_node = end
        break
    return start_node, end_node, max_dist

In [68]:
# pos = nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 0)
# #nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos, edgelist=G.edges())

In [69]:
fire_stations = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\casernes.csv")
fire_stations.head()

,CASERNE,NO_CIVIQUE,RUE,LATITUDE,LONGITUDE,ARRONDISSEMENT,VILLE,DATE_DEBUT,DATE_FIN,MTM8_X,MTM8_Y
0,3,256,rue Young,45.493454,-73.560172,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,300097.0,5039283.0
1,15,1255,rue de la Sucrerie,45.484419,-73.560917,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,300038.0,5038279.0
2,23,523,place Saint-Henri,45.477925,-73.585251,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,298135.0,5037559.0
3,33,6040,boulevard Monk,45.457841,-73.595450,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,297335.0,5035328.0
4,9,8100,boulevard Saint-Michel,45.563748,-73.610169,VILLERAY-SAINT-MICHEL-PARC-EXTENSION,NaN,2019-05-06T07:00:00,NaN,296200.0,5047099.0


In [70]:
fs_update = dict(zip(G.nodes, [{'FIRE_STATION': False} for x in G.nodes]))
nx.set_node_attributes(G, fs_update)
for index, row in fire_stations.iterrows():
    min_dist = 1000
    f_long, f_lat = row['LONGITUDE'], row['LATITUDE']
    for node in list(G.nodes):
        dist = distance_in_km(node, (f_long, f_lat))
        if dist < min_dist:
            min_dist = dist
            closest_node = node
    nx.set_node_attributes(G, {closest_node: {'FIRE_STATION': True}})

In [86]:
fs_nodes = [x for x, y in G.nodes(data=True) if y['FIRE_STATION']==True]
junction_nodes = [x for x in G.nodes if len(list(G.edges(x))) > 2]
idx = 0
for node in junction_nodes:
    idx += 1
    print(idx, '/', len(junction_nodes))
    min_dist = 1000
    for fs in fs_nodes:
        try:
            dist = nx.shortest_path_length(G, source=fs, target=node, weight='time')
        except nx.NetworkXNoPath:
            continue
        if dist < min_dist:
            min_dist = dist
    nx.set_node_attributes(G, {node: {'TIME_TO_FS': min_dist}})

1 / 15071
2 / 15071
3 / 15071
4 / 15071
5 / 15071
6 / 15071
7 / 15071
8 / 15071
9 / 15071
10 / 15071
11 / 15071
12 / 15071
13 / 15071
14 / 15071
15 / 15071
16 / 15071
17 / 15071
18 / 15071
19 / 15071
20 / 15071
21 / 15071
22 / 15071
23 / 15071
24 / 15071
25 / 15071
26 / 15071
27 / 15071
28 / 15071
29 / 15071
30 / 15071
31 / 15071
32 / 15071
33 / 15071
34 / 15071
35 / 15071
36 / 15071
37 / 15071
38 / 15071
39 / 15071
40 / 15071
41 / 15071
42 / 15071
43 / 15071
44 / 15071
45 / 15071
46 / 15071
47 / 15071
48 / 15071
49 / 15071
50 / 15071
51 / 15071
52 / 15071
53 / 15071
54 / 15071
55 / 15071
56 / 15071
57 / 15071
58 / 15071
59 / 15071
60 / 15071
61 / 15071
62 / 15071
63 / 15071
64 / 15071
65 / 15071
66 / 15071
67 / 15071
68 / 15071
69 / 15071
70 / 15071
71 / 15071
72 / 15071
73 / 15071
74 / 15071
75 / 15071
76 / 15071
77 / 15071
78 / 15071
79 / 15071
80 / 15071
81 / 15071
82 / 15071
83 / 15071
84 / 15071
85 / 15071
86 / 15071
87 / 15071
88 / 15071
89 / 15071
90 / 15071
91 / 15071
92 / 150

693 / 15071
694 / 15071
695 / 15071
696 / 15071
697 / 15071
698 / 15071
699 / 15071
700 / 15071
701 / 15071
702 / 15071
703 / 15071
704 / 15071
705 / 15071
706 / 15071
707 / 15071
708 / 15071
709 / 15071
710 / 15071
711 / 15071
712 / 15071
713 / 15071
714 / 15071
715 / 15071
716 / 15071
717 / 15071
718 / 15071
719 / 15071
720 / 15071
721 / 15071
722 / 15071
723 / 15071
724 / 15071
725 / 15071
726 / 15071
727 / 15071
728 / 15071
729 / 15071
730 / 15071
731 / 15071
732 / 15071
733 / 15071
734 / 15071
735 / 15071
736 / 15071
737 / 15071
738 / 15071
739 / 15071
740 / 15071
741 / 15071
742 / 15071
743 / 15071
744 / 15071
745 / 15071
746 / 15071
747 / 15071
748 / 15071
749 / 15071
750 / 15071
751 / 15071
752 / 15071
753 / 15071
754 / 15071
755 / 15071
756 / 15071
757 / 15071
758 / 15071
759 / 15071
760 / 15071
761 / 15071
762 / 15071
763 / 15071
764 / 15071
765 / 15071
766 / 15071
767 / 15071
768 / 15071
769 / 15071
770 / 15071
771 / 15071
772 / 15071
773 / 15071
774 / 15071
775 / 15071
776 

1347 / 15071
1348 / 15071
1349 / 15071
1350 / 15071
1351 / 15071
1352 / 15071
1353 / 15071
1354 / 15071
1355 / 15071
1356 / 15071
1357 / 15071
1358 / 15071
1359 / 15071
1360 / 15071
1361 / 15071
1362 / 15071
1363 / 15071
1364 / 15071
1365 / 15071
1366 / 15071
1367 / 15071
1368 / 15071
1369 / 15071
1370 / 15071
1371 / 15071
1372 / 15071
1373 / 15071
1374 / 15071
1375 / 15071
1376 / 15071
1377 / 15071
1378 / 15071
1379 / 15071
1380 / 15071
1381 / 15071
1382 / 15071
1383 / 15071
1384 / 15071
1385 / 15071
1386 / 15071
1387 / 15071
1388 / 15071
1389 / 15071
1390 / 15071
1391 / 15071
1392 / 15071
1393 / 15071
1394 / 15071
1395 / 15071
1396 / 15071
1397 / 15071
1398 / 15071
1399 / 15071
1400 / 15071
1401 / 15071
1402 / 15071
1403 / 15071
1404 / 15071
1405 / 15071
1406 / 15071
1407 / 15071
1408 / 15071
1409 / 15071
1410 / 15071
1411 / 15071
1412 / 15071
1413 / 15071
1414 / 15071
1415 / 15071
1416 / 15071
1417 / 15071
1418 / 15071
1419 / 15071
1420 / 15071
1421 / 15071
1422 / 15071
1423 / 15071

1978 / 15071
1979 / 15071
1980 / 15071
1981 / 15071
1982 / 15071
1983 / 15071
1984 / 15071
1985 / 15071
1986 / 15071
1987 / 15071
1988 / 15071
1989 / 15071
1990 / 15071
1991 / 15071
1992 / 15071
1993 / 15071
1994 / 15071
1995 / 15071
1996 / 15071
1997 / 15071
1998 / 15071
1999 / 15071
2000 / 15071
2001 / 15071
2002 / 15071
2003 / 15071
2004 / 15071
2005 / 15071
2006 / 15071
2007 / 15071
2008 / 15071
2009 / 15071
2010 / 15071
2011 / 15071
2012 / 15071
2013 / 15071
2014 / 15071
2015 / 15071
2016 / 15071
2017 / 15071
2018 / 15071
2019 / 15071
2020 / 15071
2021 / 15071
2022 / 15071
2023 / 15071
2024 / 15071
2025 / 15071
2026 / 15071
2027 / 15071
2028 / 15071
2029 / 15071
2030 / 15071
2031 / 15071
2032 / 15071
2033 / 15071
2034 / 15071
2035 / 15071
2036 / 15071
2037 / 15071
2038 / 15071
2039 / 15071
2040 / 15071
2041 / 15071
2042 / 15071
2043 / 15071
2044 / 15071
2045 / 15071
2046 / 15071
2047 / 15071
2048 / 15071
2049 / 15071
2050 / 15071
2051 / 15071
2052 / 15071
2053 / 15071
2054 / 15071

2609 / 15071
2610 / 15071
2611 / 15071
2612 / 15071
2613 / 15071
2614 / 15071
2615 / 15071
2616 / 15071
2617 / 15071
2618 / 15071
2619 / 15071
2620 / 15071
2621 / 15071
2622 / 15071
2623 / 15071
2624 / 15071
2625 / 15071
2626 / 15071
2627 / 15071
2628 / 15071
2629 / 15071
2630 / 15071
2631 / 15071
2632 / 15071
2633 / 15071
2634 / 15071
2635 / 15071
2636 / 15071
2637 / 15071
2638 / 15071
2639 / 15071
2640 / 15071
2641 / 15071
2642 / 15071
2643 / 15071
2644 / 15071
2645 / 15071
2646 / 15071
2647 / 15071
2648 / 15071
2649 / 15071
2650 / 15071
2651 / 15071
2652 / 15071
2653 / 15071
2654 / 15071
2655 / 15071
2656 / 15071
2657 / 15071
2658 / 15071
2659 / 15071
2660 / 15071
2661 / 15071
2662 / 15071
2663 / 15071
2664 / 15071
2665 / 15071
2666 / 15071
2667 / 15071
2668 / 15071
2669 / 15071
2670 / 15071
2671 / 15071
2672 / 15071
2673 / 15071
2674 / 15071
2675 / 15071
2676 / 15071
2677 / 15071
2678 / 15071
2679 / 15071
2680 / 15071
2681 / 15071
2682 / 15071
2683 / 15071
2684 / 15071
2685 / 15071

...

12512 / 15071
12513 / 15071
12514 / 15071
12515 / 15071
12516 / 15071
12517 / 15071
12518 / 15071
12519 / 15071
12520 / 15071
12521 / 15071
12522 / 15071
12523 / 15071
12524 / 15071
12525 / 15071
12526 / 15071
12527 / 15071
12528 / 15071
12529 / 15071
12530 / 15071
12531 / 15071
12532 / 15071
12533 / 15071
12534 / 15071
12535 / 15071
12536 / 15071
12537 / 15071
12538 / 15071
12539 / 15071
12540 / 15071
12541 / 15071
12542 / 15071
12543 / 15071
12544 / 15071
12545 / 15071
12546 / 15071
12547 / 15071
12548 / 15071
12549 / 15071
12550 / 15071
12551 / 15071
12552 / 15071
12553 / 15071
12554 / 15071
12555 / 15071
12556 / 15071
12557 / 15071
12558 / 15071
12559 / 15071
12560 / 15071
12561 / 15071
12562 / 15071
12563 / 15071
12564 / 15071
12565 / 15071
12566 / 15071
12567 / 15071
12568 / 15071
12569 / 15071
12570 / 15071
12571 / 15071
12572 / 15071
12573 / 15071
12574 / 15071
12575 / 15071
12576 / 15071
12577 / 15071
12578 / 15071
12579 / 15071
12580 / 15071
12581 / 15071
12582 / 15071
12583 

13098 / 15071
13099 / 15071
13100 / 15071
13101 / 15071
13102 / 15071
13103 / 15071
13104 / 15071
13105 / 15071
13106 / 15071
13107 / 15071
13108 / 15071
13109 / 15071
13110 / 15071
13111 / 15071
13112 / 15071
13113 / 15071
13114 / 15071
13115 / 15071
13116 / 15071
13117 / 15071
13118 / 15071
13119 / 15071
13120 / 15071
13121 / 15071
13122 / 15071
13123 / 15071
13124 / 15071
13125 / 15071
13126 / 15071
13127 / 15071
13128 / 15071
13129 / 15071
13130 / 15071
13131 / 15071
13132 / 15071
13133 / 15071
13134 / 15071
13135 / 15071
13136 / 15071
13137 / 15071
13138 / 15071
13139 / 15071
13140 / 15071
13141 / 15071
13142 / 15071
13143 / 15071
13144 / 15071
13145 / 15071
13146 / 15071
13147 / 15071
13148 / 15071
13149 / 15071
13150 / 15071
13151 / 15071
13152 / 15071
13153 / 15071
13154 / 15071
13155 / 15071
13156 / 15071
13157 / 15071
13158 / 15071
13159 / 15071
13160 / 15071
13161 / 15071
13162 / 15071
13163 / 15071
13164 / 15071
13165 / 15071
13166 / 15071
13167 / 15071
13168 / 15071
13169 

13684 / 15071
13685 / 15071
13686 / 15071
13687 / 15071
13688 / 15071
13689 / 15071
13690 / 15071
13691 / 15071
13692 / 15071
13693 / 15071
13694 / 15071
13695 / 15071
13696 / 15071
13697 / 15071
13698 / 15071
13699 / 15071
13700 / 15071
13701 / 15071
13702 / 15071
13703 / 15071
13704 / 15071
13705 / 15071
13706 / 15071
13707 / 15071
13708 / 15071
13709 / 15071
13710 / 15071
13711 / 15071
13712 / 15071
13713 / 15071
13714 / 15071
13715 / 15071
13716 / 15071
13717 / 15071
13718 / 15071
13719 / 15071
13720 / 15071
13721 / 15071
13722 / 15071
13723 / 15071
13724 / 15071
13725 / 15071
13726 / 15071
13727 / 15071
13728 / 15071
13729 / 15071
13730 / 15071
13731 / 15071
13732 / 15071
13733 / 15071
13734 / 15071
13735 / 15071
13736 / 15071
13737 / 15071
13738 / 15071
13739 / 15071
13740 / 15071
13741 / 15071
13742 / 15071
13743 / 15071
13744 / 15071
13745 / 15071
13746 / 15071
13747 / 15071
13748 / 15071
13749 / 15071
13750 / 15071
13751 / 15071
13752 / 15071
13753 / 15071
13754 / 15071
13755 

14270 / 15071
14271 / 15071
14272 / 15071
14273 / 15071
14274 / 15071
14275 / 15071
14276 / 15071
14277 / 15071
14278 / 15071
14279 / 15071
14280 / 15071
14281 / 15071
14282 / 15071
14283 / 15071
14284 / 15071
14285 / 15071
14286 / 15071
14287 / 15071
14288 / 15071
14289 / 15071
14290 / 15071
14291 / 15071
14292 / 15071
14293 / 15071
14294 / 15071
14295 / 15071
14296 / 15071
14297 / 15071
14298 / 15071
14299 / 15071
14300 / 15071
14301 / 15071
14302 / 15071
14303 / 15071
14304 / 15071
14305 / 15071
14306 / 15071
14307 / 15071
14308 / 15071
14309 / 15071
14310 / 15071
14311 / 15071
14312 / 15071
14313 / 15071
14314 / 15071
14315 / 15071
14316 / 15071
14317 / 15071
14318 / 15071
14319 / 15071
14320 / 15071
14321 / 15071
14322 / 15071
14323 / 15071
14324 / 15071
14325 / 15071
14326 / 15071
14327 / 15071
14328 / 15071
14329 / 15071
14330 / 15071
14331 / 15071
14332 / 15071
14333 / 15071
14334 / 15071
14335 / 15071
14336 / 15071
14337 / 15071
14338 / 15071
14339 / 15071
14340 / 15071
14341 

14856 / 15071
14857 / 15071
14858 / 15071
14859 / 15071
14860 / 15071
14861 / 15071
14862 / 15071
14863 / 15071
14864 / 15071
14865 / 15071
14866 / 15071
14867 / 15071
14868 / 15071
14869 / 15071
14870 / 15071
14871 / 15071
14872 / 15071
14873 / 15071
14874 / 15071
14875 / 15071
14876 / 15071
14877 / 15071
14878 / 15071
14879 / 15071
14880 / 15071
14881 / 15071
14882 / 15071
14883 / 15071
14884 / 15071
14885 / 15071
14886 / 15071
14887 / 15071
14888 / 15071
14889 / 15071
14890 / 15071
14891 / 15071
14892 / 15071
14893 / 15071
14894 / 15071
14895 / 15071
14896 / 15071
14897 / 15071
14898 / 15071
14899 / 15071
14900 / 15071
14901 / 15071
14902 / 15071
14903 / 15071
14904 / 15071
14905 / 15071
14906 / 15071
14907 / 15071
14908 / 15071
14909 / 15071
14910 / 15071
14911 / 15071
14912 / 15071
14913 / 15071
14914 / 15071
14915 / 15071
14916 / 15071
14917 / 15071
14918 / 15071
14919 / 15071
14920 / 15071
14921 / 15071
14922 / 15071
14923 / 15071
14924 / 15071
14925 / 15071
14926 / 15071
14927 

In [87]:
data = []
for node in G.nodes(data=True):
    data.append({'Longitude': node[0][0], 'Latitude': node[0][1], **node[1]})
res = pd.DataFrame(data)
res.to_csv(r"node_data_time.csv")

In [88]:
nx.write_gpickle(G,"full_graph_time2fs.gpickle")